In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:10])

gsk_1EJvXF


In [4]:
# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="예산"),
    ResponseSchema(name="rating", description="추천도(1~5점)"),
    ResponseSchema(name="activities", description="주요 활동 리스트")
]

In [5]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행지
	"duration": string  // 여행 기간
	"budget": string  // 예산
	"rating": string  // 추천도(1~5점)
	"activities": string  // 주요 활동 리스트
}
```


In [9]:
# 프롬프트 템플릿
template = """
여행에 대한 예정이나 리뷰를 분석하세요. 입력: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [10]:
# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

In [11]:
# 테스트 리뷰 데이터
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 
자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

In [12]:
# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

In [13]:
# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'activities': '해운대 바다 구경, 자갈치시장 회 먹기, 감천문화마을 구경',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4점'}
